In [1]:
! pip install selenium
! pip install beautifulsoup4

In [3]:
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
import time
from collections import defaultdict

In [4]:
from bs4 import BeautifulSoup as bs
from urllib.request import urlopen as url 
import pickle
import json
import time
import multiprocessing
import pandas as pd 
import numpy as np
import re
import collections
import pickle

In [22]:
chrome_options = webdriver.ChromeOptions()
# chrome_options.add_argument('--headless')  # Runs Chrome in headless mode
chrome_options.add_argument('--no-sandbox')  # Bypass OS security model
chrome_options.add_argument('--disable-dev-shm-usage')
chromedriver_path = "./chromedriver"

In [23]:
def class_extention_method(cls):
    def decorator(func):
        setattr(cls, func.__name__, func)
        return func
    return decorator

In [24]:
@class_extention_method(webdriver.chrome.webdriver.WebDriver)
def wait_elements_xpath(self, xpath, timeout = 5, verbose=True):
    try:
        return WebDriverWait(self,timeout).until(lambda d: d.find_elements_by_xpath(xpath))
    except:
        if verbose:
            print("find xpath timeout:",xpath)
        
@class_extention_method(webdriver.chrome.webdriver.WebDriver)
def wait_element_xpath(self, xpath, timeout = 5, verbose=True):
    try:
        return WebDriverWait(self,timeout).until(lambda d: d.find_element_by_xpath(xpath))
    except:
        if verbose:
            print("find xpath timeout:",xpath)

In [25]:
comment_info_xpath = "//div[@id='main']"
show_more_button_xpath = "//span[@class='more-button style-scope ytd-comment-renderer']"
number_of_comments = "//h2[@id='count']/yt-formatted-string[@class='count-text style-scope ytd-comments-header-renderer']"
# comment_total_amount_xpath = "//yt-formatted-string[@class='count-text style-scope ytd-comments-header-renderer']"

In [26]:
df = pd.read_excel('DataSource_80clips.xlsx')
# remove the "who" and sort the table with "comedian"
df = df.drop('Who?', axis=1)
df = df.sort_values('Comedian')
df.tail()

,Comedian,Title,Link of the transcript,Transcript (CONTENT!!!!),Video link
41,Urzila Carlson:,What No One Tells You About Homemade Porn,https://scrapsfromtheloft.com/2020/07/19/urzil...,"It makes no sense. Now, I know… I know I need ...",https://www.youtube.com/watch?v=AW5FoxJ-C5w&ab...
27,Vir Das,Losing It (2018),https://scrapsfromtheloft.com/2018/12/15/vir-d...,Every time a woman in India wears something re...,https://www.youtube.com/watch?v=qtswR_SW8S0&ab...
12,Vir Das,Losing It,https://scrapsfromtheloft.com/2018/12/15/vir-d...,I lost 80% of my religion this year. And I thi...,https://www.youtube.com/watch?v=wE9OuVw2XI8
17,Whitney Cummings,Marriage & Sex,https://scrapsfromtheloft.com/2017/12/25/whitn...,Love makes you think about things you never th...,https://www.youtube.com/watch?v=T6GstyTrLl0
14,Whitney Cummings,I'm Your Girlfriend: Women Paying,https://scrapsfromtheloft.com/2017/12/25/whitn...,We’re living in a fascinating time. We’re livi...,https://www.youtube.com/watch?v=A1OKxp91B2M


In [27]:
all_urls = df["Video link"]
comedians = df.Comedian

In [28]:
len(all_urls)

80

In [29]:
len(comedians)

80

In [17]:
web_list = []
with webdriver.Chrome(chromedriver_path,options=chrome_options) as driver:    
    for url in all_urls:
        all_comments_info = []        
        prev_offest = -1
        offset = 0
        
        print(f"\nUrl: {url}")
        driver.get(url)
        # Scroll until the number of comments shows up
        while driver.wait_elements_xpath(number_of_comments, timeout=1, verbose=False) == None:
            driver.execute_script("scrollBy(0, 50)")
            
        # Scarpe comments until all the comments are collected
        while offset != prev_offest:
            prev_offest = offset
            driver.execute_script("scrollBy(0, 1000)")
            time.sleep(2)
            offset = driver.execute_script("return window.pageYOffset;");
            all_comments_info = driver.wait_elements_xpath(comment_info_xpath)
            print("", end=".")
        # Get whole html for us to find sth later
        html = driver.page_source    
        whole_page = bs(html,"html.parser")
        web_list.append(whole_page)


Url: https://www.youtube.com/watch?v=gvSqKSkRFuM&ab_channel=HBO
..........................................
Url: https://www.youtube.com/watch?v=obphkbG5FIo&ab_channel=DryBarComedy
..........................
Url: https://www.youtube.com/watch?v=EmzPp33kl7o
............
Url: https://www.youtube.com/watch?v=c_KmV-hquvc&ab_channel=NetflixIsAJoke
...........
Url: https://www.youtube.com/watch?v=oTHpQjE1VQE&ab_channel=NetflixIsAJoke
.................................
Url: https://www.youtube.com/watch?v=lWA-xsAVzgE
.............
Url: https://www.youtube.com/watch?v=X8Ug3-_S174&ab_channel=NetflixIsAJoke
...........................................
Url: https://www.youtube.com/watch?v=io2d_cpoLDg&ab_channel=NetflixIsAJoke
.......................................
Url: https://www.youtube.com/watch?v=Oa6X6ufoiBM&ab_channel=GrettaVonPissflaps
.........................................
Url: https://www.youtube.com/watch?v=RRLf-NaMfLc&list=PLVXN0tFUddVTkbmqYUTS_OCkMF2ab-NpX&index=4&ab_channel=dncZTV
.

In [18]:
len(web_list)

80

In [29]:
# get clips title
def get_titles(web_list):
    clip_title = []
    for webpage in web_list:
        title = webpage.find('yt-formatted-string',{'class': 'style-scope ytd-video-primary-info-renderer'}).getText()
        clip_title.append(title)
    return clip_title

In [68]:
def html_to_list(web_list):
    df = pd.DataFrame(columns=['Title','Thumbsup', 'Name','Comment'])    
    for web in web_list:        
        # title
        title = web.find('yt-formatted-string',{'class': 'style-scope ytd-video-primary-info-renderer'}).getText()
        
        comments = web.find_all('div', {'id':'main', 'class': 'style-scope ytd-comment-renderer'})
        comments_names_list = []
        comments_contents_list = []
        comments_thumbsup_list = []
            
        for comment in comments:
            row = {}  
            row['Title'] = title
            # reviewer's thumbsup amount
            thumbsup = comment.find('span', {'id':'vote-count-middle', 'class':'style-scope ytd-comment-action-buttons-renderer'}).getText()
            new_thumbsup = thumbsup.strip('\n ').replace('K','*1000')
            row['Thumbsup'] = int(eval(new_thumbsup))
            # reviewer's name
            name = comment.find('span', {'class': 'style-scope ytd-comment-renderer'}).getText()        
            row['Name'] = name.strip('\n ')
            # reviewer's comment
            content = comment.find('yt-formatted-string', {'id': 'content-text'}).getText()
            row['Comment'] = content
            # append to dataframe
            df = df.append(row, ignore_index=True)
    
    return df

In [70]:
df = html_to_list(web_list)
df

,Title,Thumbsup,Name,Comment
0,Ramy Youssef: Feelings (2019) | Why Do They Ma...,2400,Kenza B.,"He touched something really deep, first he ddn..."
1,Ramy Youssef: Feelings (2019) | Why Do They Ma...,1500,LMivha,How this man got his point across is absolute ...
2,Ramy Youssef: Feelings (2019) | Why Do They Ma...,1200,Marissa Urias,"Wow, people in the comments really don't get t..."
3,Ramy Youssef: Feelings (2019) | Why Do They Ma...,909,Riaz Uddin,"This is what I tell people all the time, just ..."
4,Ramy Youssef: Feelings (2019) | Why Do They Ma...,837,Shannon dobbs,They don't ask NUNS THAT! 🙄
...,...,...,...,...
17748,Black Music Tells You Everything You Need to K...,0,357 Magnum,I'm guessing he doesn't live in a colorblind w...
17749,Black Music Tells You Everything You Need to K...,3,Erin James,Then leave my dear friend. Youve been free for...
17750,Black Music Tells You Everything You Need to K...,0,I'm black So I'm slow,We didn't create the blue's!... We didn't crea...
17751,Black Music Tells You Everything You Need to K...,2,ralph salotto,"Now that you mention it he's right, the Blues ..."


In [71]:
df.to_csv('backup_scrape_result_80.csv',index=False)

In [72]:
df.to_pickle('./pickles/scrape_result_80.pkl')

In [80]:
df_name = df.groupby(["Name"], sort=True).agg(['count']).Title.sort_values('count', ascending=False)
df_name.head(100)

,count
Name,
whitepride1004,31
Sumukh,7
Michael,6
Green Envy,6
Audrey New,6
...,...
Bi- Han,3
Lydia Waweru-Morgan,3
James Bond,3


# Insert Commedians back to the dataframe

In [40]:
df = pd.read_pickle('./pickles/scrape_result_80.pkl')
df

,Title,Thumbsup,Name,Comment
0,Ramy Youssef: Feelings (2019) | Why Do They Ma...,2400,Kenza B.,"He touched something really deep, first he ddn..."
1,Ramy Youssef: Feelings (2019) | Why Do They Ma...,1500,LMivha,How this man got his point across is absolute ...
2,Ramy Youssef: Feelings (2019) | Why Do They Ma...,1200,Marissa Urias,"Wow, people in the comments really don't get t..."
3,Ramy Youssef: Feelings (2019) | Why Do They Ma...,909,Riaz Uddin,"This is what I tell people all the time, just ..."
4,Ramy Youssef: Feelings (2019) | Why Do They Ma...,837,Shannon dobbs,They don't ask NUNS THAT! 🙄
...,...,...,...,...
17748,Black Music Tells You Everything You Need to K...,0,357 Magnum,I'm guessing he doesn't live in a colorblind w...
17749,Black Music Tells You Everything You Need to K...,3,Erin James,Then leave my dear friend. Youve been free for...
17750,Black Music Tells You Everything You Need to K...,0,I'm black So I'm slow,We didn't create the blue's!... We didn't crea...
17751,Black Music Tells You Everything You Need to K...,2,ralph salotto,"Now that you mention it he's right, the Blues ..."


In [41]:
unique_titles = {}
# find all unique titles (cannot use set because it will change its order)
for t in df.Title:
    if t not in unique_titles:
        unique_titles[t] = 1
unique_titles = list(unique_titles.keys())

In [42]:
df['comedian_id'] = [unique_titles.index(t) for t in df.Title]
df

,Title,Thumbsup,Name,Comment,comedian_id
0,Ramy Youssef: Feelings (2019) | Why Do They Ma...,2400,Kenza B.,"He touched something really deep, first he ddn...",0
1,Ramy Youssef: Feelings (2019) | Why Do They Ma...,1500,LMivha,How this man got his point across is absolute ...,0
2,Ramy Youssef: Feelings (2019) | Why Do They Ma...,1200,Marissa Urias,"Wow, people in the comments really don't get t...",0
3,Ramy Youssef: Feelings (2019) | Why Do They Ma...,909,Riaz Uddin,"This is what I tell people all the time, just ...",0
4,Ramy Youssef: Feelings (2019) | Why Do They Ma...,837,Shannon dobbs,They don't ask NUNS THAT! 🙄,0
...,...,...,...,...,...
17748,Black Music Tells You Everything You Need to K...,0,357 Magnum,I'm guessing he doesn't live in a colorblind w...,79
17749,Black Music Tells You Everything You Need to K...,3,Erin James,Then leave my dear friend. Youve been free for...,79
17750,Black Music Tells You Everything You Need to K...,0,I'm black So I'm slow,We didn't create the blue's!... We didn't crea...,79
17751,Black Music Tells You Everything You Need to K...,2,ralph salotto,"Now that you mention it he's right, the Blues ...",79


In [44]:
# Append the comedians
df['Comedians'] = [comedians[idx] for idx in temp_df['comedian_id']]
df

,Title,Thumbsup,Name,Comment,comedian_id,Comedians
0,Ramy Youssef: Feelings (2019) | Why Do They Ma...,2400,Kenza B.,"He touched something really deep, first he ddn...",0,Kevin Hart
1,Ramy Youssef: Feelings (2019) | Why Do They Ma...,1500,LMivha,How this man got his point across is absolute ...,0,Kevin Hart
2,Ramy Youssef: Feelings (2019) | Why Do They Ma...,1200,Marissa Urias,"Wow, people in the comments really don't get t...",0,Kevin Hart
3,Ramy Youssef: Feelings (2019) | Why Do They Ma...,909,Riaz Uddin,"This is what I tell people all the time, just ...",0,Kevin Hart
4,Ramy Youssef: Feelings (2019) | Why Do They Ma...,837,Shannon dobbs,They don't ask NUNS THAT! 🙄,0,Kevin Hart
...,...,...,...,...,...,...
17748,Black Music Tells You Everything You Need to K...,0,357 Magnum,I'm guessing he doesn't live in a colorblind w...,79,Arsenio Hall
17749,Black Music Tells You Everything You Need to K...,3,Erin James,Then leave my dear friend. Youve been free for...,79,Arsenio Hall
17750,Black Music Tells You Everything You Need to K...,0,I'm black So I'm slow,We didn't create the blue's!... We didn't crea...,79,Arsenio Hall
17751,Black Music Tells You Everything You Need to K...,2,ralph salotto,"Now that you mention it he's right, the Blues ...",79,Arsenio Hall


In [45]:
df = df.drop(['comedian_id'], axis=1)

In [46]:
df.to_pickle('./pickles/scrape_result_80_with_comedians.pkl')

# User-Comedian Matrix

In [47]:
df

,Title,Thumbsup,Name,Comment,Comedians
0,Ramy Youssef: Feelings (2019) | Why Do They Ma...,2400,Kenza B.,"He touched something really deep, first he ddn...",Kevin Hart
1,Ramy Youssef: Feelings (2019) | Why Do They Ma...,1500,LMivha,How this man got his point across is absolute ...,Kevin Hart
2,Ramy Youssef: Feelings (2019) | Why Do They Ma...,1200,Marissa Urias,"Wow, people in the comments really don't get t...",Kevin Hart
3,Ramy Youssef: Feelings (2019) | Why Do They Ma...,909,Riaz Uddin,"This is what I tell people all the time, just ...",Kevin Hart
4,Ramy Youssef: Feelings (2019) | Why Do They Ma...,837,Shannon dobbs,They don't ask NUNS THAT! 🙄,Kevin Hart
...,...,...,...,...,...
17748,Black Music Tells You Everything You Need to K...,0,357 Magnum,I'm guessing he doesn't live in a colorblind w...,Arsenio Hall
17749,Black Music Tells You Everything You Need to K...,3,Erin James,Then leave my dear friend. Youve been free for...,Arsenio Hall
17750,Black Music Tells You Everything You Need to K...,0,I'm black So I'm slow,We didn't create the blue's!... We didn't crea...,Arsenio Hall
17751,Black Music Tells You Everything You Need to K...,2,ralph salotto,"Now that you mention it he's right, the Blues ...",Arsenio Hall


In [48]:
df_clean = df[['Name', 'Comedians']]
df_clean['Comedians'] = df_clean.Comedians.apply(lambda x: x.lower().strip())
df_clean

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,Name,Comedians
0,Kenza B.,kevin hart
1,LMivha,kevin hart
2,Marissa Urias,kevin hart
3,Riaz Uddin,kevin hart
4,Shannon dobbs,kevin hart
...,...,...
17748,357 Magnum,arsenio hall
17749,Erin James,arsenio hall
17750,I'm black So I'm slow,arsenio hall
17751,ralph salotto,arsenio hall


In [49]:
df_clean.groupby('Name').count().sort_values(by='Comedians', ascending=False).head(20)

,Comedians
Name,
whitepride1004,31
Sumukh,7
Michael,6
Green Envy,6
Audrey New,6
matthew styles,6
Whyinem,6
karissa and sweettweet,6
Wuft Chan,5


In [50]:
df_clean['duplicate'] = df_clean.duplicated()
df_clean = df_clean[~df_clean['duplicate']]

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [51]:
df_clean.groupby('Name').count().sort_values(by='Comedians', ascending=False).head(20)

,Comedians,duplicate
Name,,
Sumukh,7,7
Audrey New,6,6
Whyinem,6,6
Green Envy,5,5
Adam Szasz,5,5
matthew styles,5,5
Michael,5,5
John Paul Lafferty,4,4
Wuft Chan,4,4


In [52]:
# Select the top 100 users 
users = df_clean.groupby('Name').count().sort_values(by='Comedians', ascending=False).head(100).index
users

Index(['Sumukh', 'Audrey New', 'Whyinem', 'Green Envy', 'Adam Szasz',
       'matthew styles', 'Michael', 'John Paul Lafferty', 'Wuft Chan',
       'Papa Grounds', 'Michiko Manalang', 'Xfirex Xbites', 'Panos kanos',
       'John Smith', 'R. A.', 'A', 'Tyrone Frye', 'g', 'black bird', 'Andrew',
       'R B', 'K M', 'That Guy', 'Chris young', 'A Z', 'Lxlno', 'K J',
       'rudeboymon', 'BabyChainGang', 'sillieww', 'Kevin Smith',
       'Michael Pipkin', 'Abot Acim', 'Bonafide Slacker', 'xirsamoht x',
       'Glenn Davey', 'L K', 'Mike bee', 'J', 'Andrew Wiggin',
       'Mattastic Mattattack', 'Claudia Solomon', 'Cleveland Maker',
       'ADEDOTUN AKANDE', 'Al NOSNERD', 'yew 2oob', 'Mike Jones', 'Sam', 'J C',
       'Tha chocolate god', 'Real Talk', 'S G', 'James Bond', 'Bi- Han',
       'Cynthia Copland', 'horse sabre', 'Peter E', 'Rob', 'Jennifer Bates',
       'Anton Nym', 'John Doe',
       'Comedy Central Stand-Up\n\n\n\n      \n    Comedy Central Stand-Up',
       'Dee Cohen', 'Vikr

In [53]:
unique_comedians = list(set(df_clean.Comedians))

In [54]:
df_ucm = pd.DataFrame(0, index=users, columns=unique_comedians)
df_ucm

,jo koy,kenny sebastian,iliza shlesinger,tom segura,aziz ansari,andy woodhull,vir das,michael che,trevor noah,joe rogan,...,urzila carlson:,tiffany haddish,marc maron,dave chappelle,eric andre,bill burr,jimmy o. yang,michelle wolf,jack whitehall,hasan minhaj
Name,,,,,,,,,,,,,,,,,,,,,
Sumukh,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Audrey New,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Whyinem,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Green Envy,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Adam Szasz,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Alin Y,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
seul JB,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Sanaya Sanaya,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [55]:
df_ucm.to_pickle('./pickles/user_comedian_matrix.pkl')

In [56]:
# The way to assign value into the dataframe
df_ucm.loc['Michael', 'andy woodhull'] = 10000000

In [57]:
df_ucm

,jo koy,kenny sebastian,iliza shlesinger,tom segura,aziz ansari,andy woodhull,vir das,michael che,trevor noah,joe rogan,...,urzila carlson:,tiffany haddish,marc maron,dave chappelle,eric andre,bill burr,jimmy o. yang,michelle wolf,jack whitehall,hasan minhaj
Name,,,,,,,,,,,,,,,,,,,,,
Sumukh,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Audrey New,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Whyinem,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Green Envy,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Adam Szasz,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Alin Y,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
seul JB,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Sanaya Sanaya,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
